In [1]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

global companies_set 

warnings.filterwarnings('ignore')

# 1 - Pré-Processamento

## 1.1 - Acessando planilha de dataset original

In [2]:
path = r"D:\Documentos_D\UFCG\2021.2e\TCC\Project\analise-de-indicador-tcc-ufcg\Data\indicadores_Siteware.csv"
base_df = pd.read_csv(path,sep =",")

## 1.2 - Filtrando e renomeando apenas as colunas necessárias para análise

In [3]:
df = base_df[['Indicador','Dim1_Membro_Mome','Dim2_Membro_Mome','Dim3_Membro_Mome','VLR_Ocorrencia','Tipo_Registro','Data_Ocorrencia']]
df = df[df['Tipo_Registro'] == "Apurado"]
df = df.drop(['Tipo_Registro','Indicador'],axis=1) 
df.columns = ['Empresa','Canal_Distribuicao','Mercado','Volume_Venda','Data']

## 1.3 - Tratamento e remoção das linhas com NaNs que serão desconsideradas

In [4]:
dim1_membro_mome_nan_sum = df.Empresa.isna().sum()
dim2_membro_mome_nan_sum = df.Canal_Distribuicao.isna().sum()
dim3_membro_mome_nan_sum = df.Mercado.isna().sum()
vlr_ocorrencia_nan_sum = df.Volume_Venda.isna().sum()

df = df.fillna("NaN")

### 1.3.1 - Filtrando dataframe com apenas as linhas que possuem pelo menos Empresa (dimensão 1)

In [5]:
df = df[((df.Empresa != "NaN") & (df.Canal_Distribuicao == "NaN") & (df.Mercado == "NaN"))
        |((df.Empresa != "NaN") & (df.Canal_Distribuicao != "NaN") & (df.Mercado == "NaN"))
        |((df.Empresa != "NaN") & (df.Canal_Distribuicao != "NaN") & (df.Mercado != "NaN"))]

### 1.3.2 - Removendo do dataframe linhas que possuem o "Valor de Ocorrência" vazio

In [6]:
df = df[(df.Volume_Venda != "NaN")].reset_index()

new_index = []
for i in range(len(df)):
    new_index.append(i+1)
df = df.set_index(pd.Index(new_index))
del df['index']

# 2 - Análise Exploratória dos Dados

## 2.1 - Análise das "Empresas", dos "Canais de Distribuição" e dos "Mercados"

In [7]:
empresas = df.Empresa.unique()
canais = df.Canal_Distribuicao.unique()
mercados = df.Mercado.unique()

In [8]:
df['Mercado'] = df['Mercado'].replace(['Am.Norte','Ásia','Europa','Am.Latina','Japão','Brasil'],
                                      [   'R1'   , 'R2' ,  'R3'  ,    'R4'   ,  'R5' ,  'R6'  ])

### 2.1.1 - Agrupamento de Mercados, por Canais, por Empresa

In [9]:
group_by_empresa = df.groupby(['Empresa'])
group_by_canal = df.groupby(['Canal_Distribuicao'])
group_by_mercado = df.groupby(['Mercado'])

companies_set = [] 

for company_name in empresas:
    temp_df = group_by_empresa.get_group(company_name)
    companies_set.append((company_name, temp_df))

## 2.2 - Médias de Volume de Vendas

#### - Canal de Distribuição: EE

##### --- Mercado: *R1*

In [10]:
company = companies_set[2][0]
channel = 'EE'
market = 'R1'
cmp_df_r1 = companies_set[2][1]
cmp_df_r1 = cmp_df_r1[(cmp_df_r1.Canal_Distribuicao == channel) & (cmp_df_r1.Mercado == market) ]
cmp_df_r1 = cmp_df_r1[((cmp_df_r1.Empresa != "NaN") & (cmp_df_r1.Canal_Distribuicao != "NaN") & (cmp_df_r1.Mercado != "NaN"))]
cmp_df_r1 = cmp_df_r1.sort_values(by='Volume_Venda').reset_index().drop(['index'],axis=1)
cmp_df_r1 = cmp_df_r1.drop_duplicates('Data',keep='last').sort_values(by='Data').reset_index().drop(['index'],axis=1)

print("Início dos dados:")
display(cmp_df_r1.head(1))
print("Fim dos dados:")
display(cmp_df_r1.tail(1))

means_br_ee_r1 = []
year = 2012
while year < 2022: 
    query = cmp_df_r1[((cmp_df_r1.Data >= str(year)) & (cmp_df_r1.Data < str(year+1)))]
    query['Volume_Venda'] = query['Volume_Venda'].astype(float)
    
    mean = np.round(query['Volume_Venda'].mean(),4)
    
    plt.figure(figsize = (4, 3))
    boxplot = query.boxplot(column = ['Volume_Venda'])
    
    save_path = r"D:\Documentos_D\UFCG\2021.2e\TCC\graficos\boxplot\BR" + "\\" + channel +"_" + market  + "_" + str(year) + ".png"
    plt.title(company + " - " + channel + " - " + market + " - " + str(year))
    plt.savefig(save_path)
    plt.close()
    
    means_br_ee_r1.append(mean)
    year += 1

means = means_br_ee_r1
bars = ['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
y_pos = np.arange(len(bars))
plt.figure(figsize = (6, 4))
plt.title("Média do Volume de Vendas/ano | "+ companies_set[2][0]+" - EE - R1")
plt.plot(y_pos, means)
plt.xticks(y_pos, bars)

save_path = r"D:\Documentos_D\UFCG\2021.2e\TCC\graficos\media\BR" + "\\" + channel + "_" + market + "_media" ".png"
plt.savefig(save_path)

plt.close()

Início dos dados:


,Empresa,Canal_Distribuicao,Mercado,Volume_Venda,Data
0,BR,EE,R1,0.0,2012-01-01 00:00:00.000


Fim dos dados:


,Empresa,Canal_Distribuicao,Mercado,Volume_Venda,Data
119,BR,EE,R1,21600.0,2021-12-01 00:00:00.000


##### --- Mercado: *R2*

In [11]:
company = companies_set[2][0]
channel = 'EE'
market = 'R2'

cmp_df_r2 = companies_set[2][1]
cmp_df_r2 = cmp_df_r2[(cmp_df_r2.Canal_Distribuicao == channel) & (cmp_df_r2.Mercado == market) ]
cmp_df_r2 = cmp_df_r2[((cmp_df_r2.Empresa != "NaN") & (cmp_df_r2.Canal_Distribuicao != "NaN") & (cmp_df_r2.Mercado != "NaN"))]
cmp_df_r2 = cmp_df_r2.sort_values(by='Volume_Venda').reset_index().drop(['index'],axis=1)
cmp_df_r2 = cmp_df_r2.drop_duplicates('Data',keep='last').sort_values(by='Data').reset_index().drop(['index'],axis=1)

print("Início dos dados:")
display(cmp_df_r2.head(1))
print("Fim dos dados:")
display(cmp_df_r2.tail(1))

means_br_ee_r2 = []
year = 2012
while year < 2022: 
    query = cmp_df_r2[((cmp_df_r2.Data >= str(year)) & (cmp_df_r2.Data < str(year+1)))]
    query['Volume_Venda'] = query['Volume_Venda'].astype(float)
    
    mean = np.round(query['Volume_Venda'].mean(),4)
    
    plt.figure(figsize = (4, 3))
    boxplot = query.boxplot(column = ['Volume_Venda'])
    
    save_path = r"D:\Documentos_D\UFCG\2021.2e\TCC\graficos\boxplot\BR" + "\\" + channel +"_" + market  + "_" + str(year) + ".png"
    plt.title(company + " - " + channel + " - " + market + " - " + str(year))
    plt.savefig(save_path)
    plt.close()
    
    means_br_ee_r2.append(mean)
    year += 1
    
means = means_br_ee_r2
bars = ['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
y_pos = np.arange(len(bars))
plt.figure(figsize = (6, 4))
plt.title("Média do Volume de Vendas/ano | "+ companies_set[2][0]+" - EE - R2")
plt.plot(y_pos, means)
plt.xticks(y_pos, bars)

save_path = r"D:\Documentos_D\UFCG\2021.2e\TCC\graficos\media\BR" + "\\" + channel + "_" + market + "_media" ".png"
plt.savefig(save_path)

plt.close()

Início dos dados:


,Empresa,Canal_Distribuicao,Mercado,Volume_Venda,Data
0,BR,EE,R2,18362.0,2012-01-01 00:00:00.000


Fim dos dados:


,Empresa,Canal_Distribuicao,Mercado,Volume_Venda,Data
120,BR,EE,R2,22100.0,2022-01-01 00:00:00.000


##### --- Mercado: *R3*

In [12]:
company = companies_set[2][0]
channel = 'EE'
market = 'R3'

cmp_df_r3 = companies_set[2][1]
cmp_df_r3 = cmp_df_r3[(cmp_df_r3.Canal_Distribuicao == channel) & (cmp_df_r3.Mercado == market) ]
cmp_df_r3 = cmp_df_r3[((cmp_df_r3.Empresa != "NaN") & (cmp_df_r3.Canal_Distribuicao != "NaN") & (cmp_df_r3.Mercado != "NaN"))]
cmp_df_r3 = cmp_df_r3.sort_values(by='Volume_Venda').reset_index().drop(['index'],axis=1)
cmp_df_r3 = cmp_df_r3.drop_duplicates('Data',keep='last').sort_values(by='Data').reset_index().drop(['index'],axis=1)

print("Início dos dados:")
display(cmp_df_r3.head(1))
print("Fim dos dados:")
display(cmp_df_r3.tail(1))

means_br_ee_r3 = []
year = 2012
while year < 2022:
    query = cmp_df_r3[((cmp_df_r3.Data >= str(year)) & (cmp_df_r3.Data < str(year+1)))]
    query['Volume_Venda'] = query['Volume_Venda'].astype(float)
    
    mean = np.round(query['Volume_Venda'].mean(),4)
    
    plt.figure(figsize = (4, 3))
    boxplot = query.boxplot(column = ['Volume_Venda'])
    
    save_path = r"D:\Documentos_D\UFCG\2021.2e\TCC\graficos\boxplot\BR" + "\\" + channel +"_" + market  + "_" + str(year) + ".png"
    plt.title(company + " - " + channel + " - " + market + " - " + str(year))
    plt.savefig(save_path)
    plt.close()
    
    means_br_ee_r3.append(mean)
    year += 1
    
#--- Plotando o Gráfico ---#
means = means_br_ee_r3
bars = ['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
y_pos = np.arange(len(bars))
plt.figure(figsize = (6, 4))
plt.title("Média do Volume de Vendas/ano | "+ companies_set[2][0]+" - EE - R3")
plt.plot(y_pos, means)
plt.xticks(y_pos, bars)

save_path = r"D:\Documentos_D\UFCG\2021.2e\TCC\graficos\media\BR" + "\\" + channel + "_" + market + "_media" ".png"
plt.savefig(save_path)

plt.close()

Início dos dados:


,Empresa,Canal_Distribuicao,Mercado,Volume_Venda,Data
0,BR,EE,R3,44852.0,2012-01-01 00:00:00.000


Fim dos dados:


,Empresa,Canal_Distribuicao,Mercado,Volume_Venda,Data
120,BR,EE,R3,57880.0,2022-01-01 00:00:00.000


##### --- Mercado: *R4*

In [13]:
company = companies_set[2][0]
channel = 'EE'
market = 'R4'

cmp_df_r4 = companies_set[2][1]
cmp_df_r4 = cmp_df_r4[(cmp_df_r4.Canal_Distribuicao == channel) & (cmp_df_r4.Mercado == market) ]
cmp_df_r4 = cmp_df_r4[((cmp_df_r4.Empresa != "NaN") & (cmp_df_r4.Canal_Distribuicao != "NaN") & (cmp_df_r4.Mercado != "NaN"))]
cmp_df_r4 = cmp_df_r4.sort_values(by='Volume_Venda').reset_index().drop(['index'],axis=1)
cmp_df_r4 = cmp_df_r4.drop_duplicates('Data',keep='last').sort_values(by='Data').reset_index().drop(['index'],axis=1)

print("Início dos dados:")
display(cmp_df_r4.head(1))
print("Fim dos dados:")
display(cmp_df_r4.tail(1))

means_br_ee_r4 = []
year = 2012
while year < 2016: # So tem ate 01 Janeiro de 2016. Desconsiderei 2016;
    query = cmp_df_r4[((cmp_df_r4.Data >= str(year)) & (cmp_df_r4.Data < str(year+1)))]
    query['Volume_Venda'] = query['Volume_Venda'].astype(float)
    
    mean = np.round(query['Volume_Venda'].mean(),4)
    
    plt.figure(figsize = (4, 3))
    boxplot = query.boxplot(column = ['Volume_Venda'])
    
    save_path = r"D:\Documentos_D\UFCG\2021.2e\TCC\graficos\boxplot\BR" + "\\" + channel +"_" + market  + "_" + str(year) + ".png"
    plt.title(company + " - " + channel + " - " + market + " - " + str(year))
    plt.savefig(save_path)
    plt.close()

    means_br_ee_r4.append(mean)
    year += 1
    
means = means_br_ee_r4
bars = ['2012','2013','2014','2015']
y_pos = np.arange(len(bars))
plt.title("Média do Volume de Vendas/ano | "+ companies_set[2][0]+" - EE - R4")
plt.plot(y_pos, means)
plt.xticks(y_pos, bars)

save_path = r"D:\Documentos_D\UFCG\2021.2e\TCC\graficos\media\BR" + "\\" + channel + "_" + market + "_media" ".png"
plt.savefig(save_path)

plt.close()

Início dos dados:


,Empresa,Canal_Distribuicao,Mercado,Volume_Venda,Data
0,BR,EE,R4,624.0,2012-01-01 00:00:00.000


Fim dos dados:


,Empresa,Canal_Distribuicao,Mercado,Volume_Venda,Data
106,BR,EE,R4,0.0,2021-01-01 00:00:00.000


##### --- Mercado: *R5*

In [14]:
company = companies_set[2][0]
channel = 'EE'
market = 'R5'

cmp_df_r5 = companies_set[2][1]
cmp_df_r5 = cmp_df_r5[(cmp_df_r5.Canal_Distribuicao == channel) & (cmp_df_r5.Mercado == market)]
cmp_df_r5 = cmp_df_r5[((cmp_df_r5.Empresa != "NaN") & (cmp_df_r5.Canal_Distribuicao != "NaN") & (cmp_df_r5.Mercado != "NaN"))] 
cmp_df_r5 = cmp_df_r5.sort_values(by='Volume_Venda').reset_index().drop(['index'],axis=1)
cmp_df_r5 = cmp_df_r5.drop_duplicates('Data',keep='last').sort_values(by='Data').reset_index().drop(['index'],axis=1)

print("Início dos dados:")
display(cmp_df_r5.head(1))
print("Fim dos dados:")
display(cmp_df_r5.tail(1))

means_br_ee_r5 = []
year = 2012
while year < 2022:
    query = cmp_df_r5[((cmp_df_r5.Data >= str(year)) & (cmp_df_r5.Data < str(year+1)))]
    query['Volume_Venda'] = query['Volume_Venda'].astype(float)
    
    mean = np.round(query['Volume_Venda'].mean(),4)
    
    plt.figure(figsize = (4, 3))
    boxplot = query.boxplot(column = ['Volume_Venda'])
    
    save_path = r"D:\Documentos_D\UFCG\2021.2e\TCC\graficos\boxplot\BR" + "\\" + channel +"_" + market  + "_" + str(year) + ".png"
    plt.title(company + " - " + channel + " - " + market + " - " + str(year))
    plt.savefig(save_path)
    plt.close()
    
    means_br_ee_r5.append(mean)
    year += 1

means = means_br_ee_r5
bars = ['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
y_pos = np.arange(len(bars))

plt.figure(figsize = (6, 4))
plt.title("Média do Volume de Vendas/ano | "+ companies_set[2][0]+" - EE - R5")
plt.plot(y_pos, means)
plt.xticks(y_pos, bars)

save_path = r"D:\Documentos_D\UFCG\2021.2e\TCC\graficos\media\BR" + "\\" + channel + "_" + market + "_media" ".png"
plt.savefig(save_path)

plt.close()

Início dos dados:


,Empresa,Canal_Distribuicao,Mercado,Volume_Venda,Data
0,BR,EE,R5,19852.0,2012-01-01 00:00:00.000


Fim dos dados:


,Empresa,Canal_Distribuicao,Mercado,Volume_Venda,Data
119,BR,EE,R5,18860.0,2021-12-01 00:00:00.000


##### - *Correlação EE X R1 X R2 X R3 x R4 x R5*

In [15]:
cmp_df = companies_set[2][1].reset_index().drop(['index'],axis=1)
cmp_df = cmp_df[(cmp_df.Canal_Distribuicao == 'EE')]
cmp_df = cmp_df[((cmp_df.Empresa != "NaN") & (cmp_df.Canal_Distribuicao != "NaN") & (cmp_df.Mercado == "NaN"))]
cmp_df = cmp_df.sort_values(by='Volume_Venda', ascending=True)
cmp_df = cmp_df.drop_duplicates('Data',keep='last').reset_index().drop(['index'],axis=1)
cmp_df = cmp_df.sort_values(by='Data').reset_index().drop(['index'],axis=1)

new_df = pd.DataFrame()
new_df['Volume_Venda_R1'] = cmp_df_r1['Volume_Venda']
new_df['Volume_Venda_R2'] = cmp_df_r2['Volume_Venda']
new_df['Volume_Venda_R3'] = cmp_df_r3['Volume_Venda']
new_df['Volume_Venda_R4'] = cmp_df_r4['Volume_Venda']
new_df['Volume_Venda_R5'] = cmp_df_r5['Volume_Venda']
new_df['Volume_Venda_EE'] = cmp_df['Volume_Venda']
new_df['Data'] = cmp_df['Data']
new_df = new_df[((new_df.Volume_Venda_R1 != 0) | (new_df.Volume_Venda_R2 != 0) | (new_df.Volume_Venda_R3 != 0) |
                (new_df.Volume_Venda_R4 != 0) | (new_df.Volume_Venda_R5 != 0) | (new_df.Volume_Venda_EE != 0))]
new_df = new_df.drop(['Data'],axis=1)
new_df = new_df.astype(float)

plt.figure(figsize = (12, 8))
plt.title("Matriz de Confusão: EE x R1 x R2 x R3 x R4 x R5")
sns.heatmap(new_df.corr())

save_path = r"D:\Documentos_D\UFCG\2021.2e\TCC\graficos\correlacao" + "\\" + company + "\\" + channel + "_" + "_matriz_de_confusao" ".png"
plt.savefig(save_path)

plt.close()

#### - Canal de Distribuição: ME

##### --- Mercado: *No region*

In [16]:
company = companies_set[2][0]
channel = 'ME'
market = 'NaN'

cmp_df_nr = companies_set[2][1]
cmp_df_nr = cmp_df_nr[((cmp_df_nr.Canal_Distribuicao == channel) & (cmp_df_nr.Mercado == market))]
cmp_df_nr = cmp_df_nr[((cmp_df_nr.Empresa != "NaN") & (cmp_df_nr.Canal_Distribuicao != "NaN") & (cmp_df_nr.Mercado == "NaN"))] 
cmp_df_nr = cmp_df_nr.sort_values(by='Volume_Venda').reset_index().drop(['index'],axis=1)
cmp_df_nr = cmp_df_nr.drop_duplicates('Data',keep='last').sort_values(by='Data').reset_index().drop(['index'],axis=1)

print("Início dos dados:")
display(cmp_df_nr.head(1))
print("Fim dos dados:")
display(cmp_df_nr.tail(1))

means_br_me = []
year = 2012
while year < 2022:
    query = cmp_df_nr[((cmp_df_nr.Data >= str(year)) & (cmp_df_nr.Data < str(year+1)))]
    query['Volume_Venda'] = query['Volume_Venda'].astype(float)
    
    mean = np.round(query['Volume_Venda'].mean(),4)
    
    plt.figure(figsize = (4, 3))
    boxplot = query.boxplot(column = ['Volume_Venda'])
    
    market = "sem_regiao"
    
    save_path = r"D:\Documentos_D\UFCG\2021.2e\TCC\graficos\boxplot\BR" + "\\" + channel +"_" + market  + "_" + str(year) + ".png"
    plt.title(company + " - " + channel + " - " + market + " - " + str(year))
    plt.savefig(save_path)
    plt.close()
    
    means_br_me.append(mean)
    year += 1

means = means_br_me
bars = ['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
y_pos = np.arange(len(bars))
plt.figure(figsize = (6, 4))
plt.title("Média do Volume de Vendas/ano | " + companies_set[2][0]+" - ME - sem região")
plt.plot(y_pos, means)
plt.xticks(y_pos, bars)

save_path = r"D:\Documentos_D\UFCG\2021.2e\TCC\graficos\media\BR" + "\\" + channel + "_" + market + "_media" ".png"
plt.savefig(save_path)

plt.close()

Início dos dados:


,Empresa,Canal_Distribuicao,Mercado,Volume_Venda,Data
0,BR,ME,NaN,83690.0,2012-01-01 00:00:00.000


Fim dos dados:


,Empresa,Canal_Distribuicao,Mercado,Volume_Venda,Data
120,BR,ME,NaN,79980.0,2022-01-01 00:00:00.000


#### - Canal de Distribuição: MI

##### --- Mercado: *R6*

In [17]:
company = companies_set[2][0]
channel = 'MI'
market = 'R6'

cmp_df_r6 = companies_set[2][1].reset_index().drop(['index'],axis=1)
cmp_df_r6 = cmp_df_r6[((cmp_df_r6.Canal_Distribuicao == channel) & (cmp_df_r6.Mercado == market))]
cmp_df_r6 = cmp_df_r6[((cmp_df_r6.Empresa != "NaN") & (cmp_df_r6.Canal_Distribuicao != "NaN") & (cmp_df_r6.Mercado != "NaN"))]
cmp_df_r6 = cmp_df_r6.sort_values(by='Volume_Venda').reset_index().drop(['index'],axis=1)
cmp_df_r6 = cmp_df_r6.drop_duplicates('Data',keep='last').sort_values(by='Data').reset_index().drop(['index'],axis=1)

print("Início dos dados:")
display(cmp_df_r6.head(1))
print("Fim dos dados:")
display(cmp_df_r6.tail(1))

means_br_mi_r6 = []
year = 2012
while year < 2022:
    query = cmp_df_r6[((cmp_df_r6.Data >= str(year)) & (cmp_df_r6.Data < str(year+1)))]
    query['Volume_Venda'] = query['Volume_Venda'].astype(float)
    
    mean = np.round(query['Volume_Venda'].mean(),4)
    
    plt.figure(figsize = (4, 3))
    boxplot = query.boxplot(column = ['Volume_Venda'])
    
    save_path = r"D:\Documentos_D\UFCG\2021.2e\TCC\graficos\boxplot\BR" + "\\" + channel +"_" + market  + "_" + str(year) + ".png"
    plt.title(company + " - " + channel + " - " + market + " - " + str(year))
    plt.savefig(save_path)
    plt.close()
    
    means_br_mi_r6.append(mean)
    year += 1

means = means_br_mi_r6
bars = ['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
y_pos = np.arange(len(bars))
plt.figure(figsize = (6, 4))
plt.title("Média do Volume de Vendas/ano | "+ companies_set[2][0]+" - MI - R6")
plt.plot(y_pos, means)
plt.xticks(y_pos, bars)

save_path = r"D:\Documentos_D\UFCG\2021.2e\TCC\graficos\media\BR" + "\\" + channel + "_" + market + "_media" ".png"
plt.savefig(save_path)

plt.close()

Início dos dados:


,Empresa,Canal_Distribuicao,Mercado,Volume_Venda,Data
0,BR,MI,R6,4598.001,2012-01-01 00:00:00.000


Fim dos dados:


,Empresa,Canal_Distribuicao,Mercado,Volume_Venda,Data
120,BR,MI,R6,1818.0,2022-01-01 00:00:00.000


In [20]:
cmp_df_r1.sort_values(by="Volume_Venda")

,Empresa,Canal_Distribuicao,Mercado,Volume_Venda,Data
0,BR,EE,R1,0.0,2012-01-01 00:00:00.000
105,BR,EE,R1,0.0,2020-10-01 00:00:00.000
88,BR,EE,R1,0.0,2019-05-01 00:00:00.000
78,BR,EE,R1,0.0,2018-07-01 00:00:00.000
6,BR,EE,R1,0.0,2012-07-01 00:00:00.000
...,...,...,...,...,...
92,BR,EE,R1,24000.0,2019-09-01 00:00:00.000
89,BR,EE,R1,24500.0,2019-06-01 00:00:00.000
5,BR,EE,R1,25530.0,2012-06-01 00:00:00.000
11,BR,EE,R1,27000.0,2012-12-01 00:00:00.000
